# Scienaptic Data Science Assignment
Author: Sharmeen Malik

Date: May 18, 2020

In [1]:
#import modules and read file into dataframe
import pandas as pd
import matplotlib.pyplot as plt 


pd.options.display.max_columns=999
pd.options.display.max_rows=3000

data = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

## What is the account roll forward rate for the months of Mar, Apr, May and June for TB0?

In [2]:
#assign data to new dataframe and display keys
df = data
df.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

**Assumptions made about data:**
- 'Loan_Status_MartoMay' - gives data for loans that were TBO each month.
- 'Loan_Status_AprtoJun' - gives data for the next month for loans that were TBO each month in 'Loan_Status_MartoMay'.

**Methodology:**
- Roll forward rate is calculated as accounts turning TB0 in Mar, Apr and May from the original account list of 40,339 loans.
- Roll forward rate for June TBO cannot be calculated as the June data represents the status of accounts that were TB0 in May.

In [3]:
#creates stratums for data based on month
mar_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'March']
apr_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'April']
may_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'May']
marapr_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'April']
aprmay_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'May']
mayjun_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'June']

In [4]:
#QC checking to see if our assumption is correct
print(len(mar_data) == len(marapr_data))
print(len(apr_data) == len(aprmay_data))
print(len(may_data) == len(mayjun_data))

True
True
True


In [5]:
#combines account information for each month with performance in the following month
mar_data1 = pd.merge(mar_data, marapr_data, how='left', on='Loan_id', suffixes=('_mar', '_marapr'))
apr_data1 = pd.merge(apr_data, aprmay_data, how='left', on='Loan_id', suffixes=('_apr', '_aprmay'))
may_data1 = pd.merge(may_data, mayjun_data, how='left', on='Loan_id', suffixes=('_may', '_mayjun'))

In [6]:
#merges monthly data into one dataframe
data1 = pd.merge(mar_data1, apr_data1, how='outer', on='Loan_id')
data1 = pd.merge(data1, may_data1, how='outer', on='Loan_id')

In [7]:
#QC to ensure all data has been captured
len(data1) == len(df['Loan Details'])

True

In [8]:
#displays column names
data1.columns

Index(['Loan_id', 'Bucket_mar', 'TENURE_mar', 'PRINBALANCE_mar',
       'Months on Books_mar', 'Sanctioned Amount_mar', 'Month_mar',
       'Bucket_marapr', 'PRINBALANCE_marapr', 'TENURE_marapr',
       'Months on Books_marapr', 'Sanctioned Amount_marapr', 'Month_marapr',
       'Bucket_apr', 'TENURE_apr', 'PRINBALANCE_apr', 'Months on Books_apr',
       'Sanctioned Amount_apr', 'Month_apr', 'Bucket_aprmay',
       'PRINBALANCE_aprmay', 'TENURE_aprmay', 'Months on Books_aprmay',
       'Sanctioned Amount_aprmay', 'Month_aprmay', 'Bucket_may', 'TENURE_may',
       'PRINBALANCE_may', 'Months on Books_may', 'Sanctioned Amount_may',
       'Month_may', 'Bucket_mayjun', 'PRINBALANCE_mayjun', 'TENURE_mayjun',
       'Months on Books_mayjun', 'Sanctioned Amount_mayjun', 'Month_mayjun'],
      dtype='object')

In [9]:
#displays value counts for statuses in each month
cols = ['Bucket_mar','Bucket_marapr', 'Bucket_apr', 'Bucket_aprmay', 'Bucket_may', 'Bucket_mayjun']

for c in cols:
    print(c)
    print(data1[c].value_counts())
    print('\n')

Bucket_mar
TB0    24575
Name: Bucket_mar, dtype: int64


Bucket_marapr
REGULAR    22475
TB0         1709
TB1          391
Name: Bucket_marapr, dtype: int64


Bucket_apr
TB0    27079
Name: Bucket_apr, dtype: int64


Bucket_aprmay
REGULAR    24964
TB0         1574
TB1          541
Name: Bucket_aprmay, dtype: int64


Bucket_may
TB0    29115
Name: Bucket_may, dtype: int64


Bucket_mayjun
REGULAR    27085
TB0         1577
TB1          453
Name: Bucket_mayjun, dtype: int64




In [10]:
#creates a dataframe to display bucket migration for each month
cols1 = ['Bucket_mar', 'Bucket_apr', 'Bucket_may']
tot_TBO = pd.DataFrame()
for c in cols1:
    tot_TBO[c] = data1[c].value_counts()


cols2 = ['Bucket_marapr', 'Bucket_aprmay', 'Bucket_mayjun']
migration = pd.DataFrame()
for c in cols2:
    migration[c] = data1[c].value_counts()

print('Total TBO loans:')
print(tot_TBO)
print('\n')
print('Migration of loans after one month:')
print(migration)


Total TBO loans:
     Bucket_mar  Bucket_apr  Bucket_may
TB0       24575       27079       29115


Migration of loans after one month:
         Bucket_marapr  Bucket_aprmay  Bucket_mayjun
REGULAR          22475          24964          27085
TB0               1709           1574           1577
TB1                391            541            453


In [11]:
#calculates roll forward rate for each month
mar_roll = data1['Bucket_mar'].count() / len(data1) * 100
apr_roll = data1['Bucket_apr'].count() / len(data1) * 100
may_roll = data1['Bucket_may'].count() / len(data1) * 100

perc_marapr = (apr_roll - mar_roll) / mar_roll * 100
perc_aprmay = (may_roll - apr_roll) / apr_roll * 100

print('Roll rate for TBO loans in March: ', mar_roll)
print('Roll rate for TB0 loans in April: ', apr_roll)
print('Roll rate for TB0 loans in May:   ', may_roll)
print('\n')
print('Percentage increase from March to April:   ', perc_marapr)
print('Percentage increase from April to May:     ', perc_aprmay)

Roll rate for TBO loans in March:  60.92119289025509
Roll rate for TB0 loans in April:  67.12858524009023
Roll rate for TB0 loans in May:    72.17581001016387


Percentage increase from March to April:    10.189216683621554
Percentage increase from April to May:      7.518741460172111


In [12]:
#QC checking to see whether there are more TBO loans in May than in March
len(may_data1) > len(mar_data1)

True

Looking at the roll rates for each months we can see an increasing trend in the number of accounts that rolled over. However, the percentage increase has gone down from 10.19 to 7.52.

In [13]:
#calculates roll forward rate for loans after one month for loans that were TB0 each month
mar_roll1 = round(migration.iloc[2,0] / migration['Bucket_marapr'].sum() * 100, 3)
apr_roll1 = round(migration.iloc[2,1] / migration['Bucket_aprmay'].sum() * 100, 3)
may_roll1 = round(migration.iloc[2,2] / migration['Bucket_mayjun'].sum() * 100, 3)

print('Roll rate for March TB0 loans: ', mar_roll1)
print('Roll rate for April TB0 loans: ', apr_roll1)
print('Roll rate for May TB0 loans:   ', may_roll1)

Roll rate for March TB0 loans:  1.591
Roll rate for April TB0 loans:  1.998
Roll rate for May TB0 loans:    1.556


## What is the Princple outstanding balance roll forward rate for the same months as above for TB0?

**Methodology**

Principal Balance was not provided for all loans for the beginning of March, therefore the rate was calculated against the total sanctioned amount of all loans.

In [14]:
#calculates total dollar santioned for entire portfolio
total_sanctioned = df['Loan Details']['Sanctioned Amount'].sum()

print('Total sanctioned amount: ', total_sanctioned)

Total sanctioned amount:  1862351670144


In [15]:
#calculates balance roll forward rate for each month
mar_bal_roll = data1['PRINBALANCE_mar'].sum() / total_sanctioned * 100
apr_bal_roll = data1['PRINBALANCE_apr'].sum() / total_sanctioned * 100
may_bal_roll = data1['PRINBALANCE_may'].sum() / total_sanctioned * 100

print('Balance roll for TB0 loans in March: ', data1['PRINBALANCE_mar'].sum())
print('Balance roll for TB0 loans in April: ', data1['PRINBALANCE_apr'].sum())
print('Balance roll for TB0 loans in May:   ', data1['PRINBALANCE_may'].sum())

Balance roll for TB0 loans in March:  827728346975.54
Balance roll for TB0 loans in April:  904446972331.73
Balance roll for TB0 loans in May:    994376841007.47


In [16]:
print('Balance roll rate for TB0 loans in March: ', mar_bal_roll)
print('Balance roll rate for TB0 loans in April: ', apr_bal_roll)
print('Balance roll rate for TB0 loans in May:   ', may_bal_roll)

Balance roll rate for TB0 loans in March:  44.44533007622233
Balance roll rate for TB0 loans in April:  48.56477897441339
Balance roll rate for TB0 loans in May:    53.393612868539655


In [17]:
#calculates percentage increase/decrease for roll rates
perc_marapr_bal = (apr_bal_roll - mar_bal_roll) / mar_bal_roll * 100
perc_aprmay_bal = (may_bal_roll - apr_bal_roll) / apr_bal_roll * 100

print('Percentage increase from March to April:   ', perc_marapr_bal)
print('Percentage increase from April to May:     ', perc_aprmay_bal)

Percentage increase from March to April:    9.268575328670847
Percentage increase from April to May:      9.943078082719904


Looking at the roll rates for each months we can see an increasing trend in the number of accounts that rolled over. The percentage increase has also increased from 9.27 to 9.94.

## Is there any correlation between the princple balance remaining and the term completed

In [18]:
#creates dataframe for accounts
bal_term_data = df['Loan_Status_MartoMay']

In [19]:
#checks length of dataframe
len(bal_term_data)

80769

In [20]:
#drops duplicates and keeps first instance of each account 
#checks length of dataframe
bal_term_data = bal_term_data.drop_duplicates(subset='Loan_id', keep='first')

len(bal_term_data)

40339

The dataframe now includes first instance of TB0 for all accounts

In [22]:
#calculates term completed as defined by 'month on books/tenure'
bal_term_data['term_comp'] = (bal_term_data['Months on Books'] / bal_term_data['TENURE'])

bal_term_data.head()

/Applications/anaconda3/envs/LoanStreet/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Loan_id,Bucket,TENURE,PRINBALANCE,Months on Books,Sanctioned Amount,Month,term_comp
0,A810487695,TB0,48,10539100.0,37,30000000,April,0.770833
1,A205346000,TB0,36,3693500.0,33,22000000,March,0.916667
2,A675851816,TB0,48,24353700.0,30,40000000,March,0.625000
4,A269873383,TB0,36,4441700.0,30,20000000,May,0.833333
5,A948976866,TB0,36,20002300.0,19,30000000,April,0.527778


In [ ]:
#calculates correlation between principal balance remaining and term completed
var1 = bal_term_data['term_comp']
var2 = bal_term_data['PRINBALANCE']

correlation = var1.corr(var2)
correlation

If we estimate the absolute correlation value as being moderate if it falls between 0.5 and 0.7, we can conclude that there is a moderate correlation between term completed and principal balance.  In addition as this is a negative value, we conclude that this is a negative relationship, meaning as there is an increase in one variable there is a decrease in the other and vice-versa.

## What is the average attempt intensity, RPC rate and PTP rate on accounts for the months of Mar, Apr and May

In [ ]:
df.keys()

In [ ]:
#assigns 'Call Detail' table to dataframe
call_data = df['Call Details']
call_data.tail()

In [ ]:
#groups call data by month and accounts and then aggregates
month_table = call_data.groupby(['month', 'Application_Id']).aggregate({'month':'nunique', 'Right_Party_Contact':'sum', 'Promise_to_pay':'sum', 'total_contacts':'sum'})

month_table.rename(columns={'month':'month_count'}, inplace=True)

month_table

In [ ]:
#calculates Right Party Contacts (RPC) rate and Percentage of outbound calls resulting in Promise to Pay (PTP) for each account every month

month_table['RPC_rate'] = round(month_table['Right_Party_Contact'] / month_table['total_contacts'], 3)

month_table['PTP_rate'] = round((month_table['Promise_to_pay']) / month_table['total_contacts'], 3)

month_table

In [ ]:
#displays descriptive statistics for the grouped data

month_table.describe()

Due to the high max value for 'Right_Party_Contact' and 'total_contact', let's explore further

In [ ]:
x = month_table['Right_Party_Contact']
y = month_table['total_contacts']

plt.boxplot([x,y])
plt.show()

In [ ]:
#exploring max value observation further
month_table[month_table['Right_Party_Contact']==651]

The max value looks like an extreme outlier where there might have been an input error. I will exclude this case.

In [ ]:
month_table = month_table[month_table['Right_Party_Contact']!=651]

In [ ]:
month_table.describe()

There still seem to be high values but will need to research this further for correct treatment, so will leave this in for now.

In [ ]:
#calculates average rates for each month based on account activity
avg_monthly = month_table.groupby('month').agg({'total_contacts':'mean', 'RPC_rate':'mean', 'PTP_rate':'mean'})

avg_monthly.rename(columns={'total_contacts':'avg_intensity','RPC_rate':'avg_rpc_rate', 'PTP_rate':'avg_ptp_rate'}, inplace=True)

avg_monthly

## Identify the agents who are showing the best and worst performance. 

In [ ]:
#groups data by agent and then aggregates
agent_table = call_data.groupby(['Login_ID']).aggregate({'month':'nunique', 'Right_Party_Contact':'sum', 'Promise_to_pay':'sum', 'total_contacts':'sum'})

agent_table.rename(columns={'month':'month_count'}, inplace=True)

agent_table.head()

In [ ]:
#calculates Right Party Contacts (RPC) rate and Percentage of outbound calls resulting in Promise to Pay (PTP) for each agent
agent_table['RPC_rate'] = round((agent_table['Right_Party_Contact'] / agent_table['total_contacts']), 3)

agent_table['PTP_rate'] = round(((agent_table['Promise_to_pay']) / agent_table['total_contacts']), 3)

agent_table.describe()


In [ ]:
#calculates performance metrics
agent_table['perf_metric'] = agent_table['month_count'] * agent_table['total_contacts'] * agent_table['Right_Party_Contact'] * (agent_table['Promise_to_pay'] + 0.001)

agent_table['perf_metric1'] = agent_table['month_count'] * agent_table['RPC_rate'] * (agent_table['PTP_rate'] + 0.001)

agent_table['perf_metric2'] = agent_table['month_count'] * agent_table['total_contacts'] * agent_table['RPC_rate'] * (agent_table['PTP_rate'] + 0.001)

agent_table.sort_values(by=['perf_metric1'], ascending=False).head(10)

In [ ]:
agent_table.describe()

In order to rank the agents' performance, we will use per_metric1 as the standard deviation is minimum for this metric.

In [ ]:
#calculates best and worst performance
top_five = agent_table.sort_values(by=['perf_metric1'], ascending=False).head(5)

bottom_five = agent_table.sort_values(by=['perf_metric1'], ascending=True).head(5)

In [ ]:
top_five

In [ ]:
bottom_five

For assessing agent performance, there were instances where the RPC rate was showing as 1.00 while the PTP rate was showing as 0.00. This was due to the fact that the agent made only one contact and was able to connect with the right party and was unable to get a promise to pay.  However, due to this characteristic, using either of the rates on their own would not account for such granular nuance.  There for the performance metric devised attempts to not only account for such cases but also to account for the number of months that the agent has been involved with the operation.  Specifically, the metric multiplies the number of months with the RPC rate and the PTP rate. In addition, in order to prevent a PTP rate value of 0 from being included in the calculation, an addition of 0.001 was made.